In [2]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")

In [117]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
pd.set_option('display.max_columns', None)

In [56]:
df=pd.read_csv('train.csv')

In [126]:
df_train.head()

,MachineID,ProductName,EngineVersion,AppVersion,SignatureVersion,IsBetaUser,RealTimeProtectionState,IsPassiveModeEnabled,AntivirusConfigID,NumAntivirusProductsInstalled,NumAntivirusProductsEnabled,HasTpm,CountryID,CityID,GeoRegionID,LocaleEnglishNameID,PlatformType,Processor,OSVersion,OSBuildNumber,OSProductSuite,OsPlatformSubRelease,OSBuildLab,SKUEditionName,IsSystemProtected,AutoSampleSubmissionEnabled,SMode,IEVersionID,FirewallEnabled,EnableLUA,MDC2FormFactor,DeviceFamily,OEMNameID,OEMModelID,ProcessorCoreCount,ProcessorManufacturerID,ProcessorModelID,PrimaryDiskCapacityMB,PrimaryDiskType,SystemVolumeCapacityMB,HasOpticalDiskDrive,TotalPhysicalRAMMB,ChassisType,PrimaryDisplayDiagonalInches,PrimaryDisplayResolutionHorizontal,PrimaryDisplayResolutionVertical,PowerPlatformRole,InternalBatteryNumberOfCharges,NumericOSVersion,OSArchitecture,OSBranch,OSBuildNumberOnly,OSBuildRevisionOnly,OSEdition,OSSkuFriendlyName,OSInstallType,OSInstallLanguageID,OSUILocaleID,AutoUpdateOptionsName,IsPortableOS,OSGenuineState,LicenseActivationChannel,IsFlightsDisabled,FlightRing,FirmwareManufacturerID,FirmwareVersionID,IsSecureBootEnabled,IsVirtualDevice,IsTouchEnabled,IsPenCapable,IsAlwaysOnAlwaysConnectedCapable,IsGamer,RegionIdentifier,DateAS,DateOS,target
0,f541bae429089117c4aac39c90dd3416,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1003.0,0,7,0,53447,1,1,1,51,120232,98,103,windows10,x86,10.0.0.0,14393,768,rs1,14393.2214.x86fre.rs1_release_1.180402-1758,Home,1,0,0,98,1,1,SmallTablet,Windows.Desktop,561,330367,4,5,1850,15028,SSD,14348,0,1024,Notebook,8,800,1280,Slate,1.420000e+02,10.0.14393.2214,x86,rs1_release,14393,2214,Core,CORE,Update,5,26,UNKNOWN,0,IS_GENUINE,Retail,0,Retail,513,21964,0,0,1,0,1,0,6,2018-09-10 10:11:00,2018-04-17,0
1,dc2b14d9ce3a0ce4050bb640190f2ca5,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1465.0,0,7,0,53447,1,1,1,141,112854,167,227,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1,0,0,137,1,1,AllInOne,Windows.Desktop,2668,25212,4,5,2407,953869,HDD,952592,1,4096,AllinOne,19,1600,900,Desktop,4.294967e+09,10.0.17134.228,amd64,rs4_release,17134,228,Professional,PROFESSIONAL,UUPUpgrade,9,34,FullAuto,0,IS_GENUINE,OEM:DM,0,Retail,628,44548,1,0,0,0,0,0,10,2018-08-16 00:01:00,2018-08-14,1
2,fd20c5f010e9c5f91ad1c6b3e0da68a0,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1546.0,0,7,0,53447,1,1,1,51,41759,98,103,windows10,x64,10.0.0.0,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1,0,0,137,1,1,Desktop,Windows.Desktop,3035,263666,4,5,2719,228936,SSD,228321,1,8192,Desktop,24,1920,1080,Desktop,4.294967e+09,10.0.17134.285,amd64,rs4_release,17134,285,Core,CORE,Reset,5,26,FullAuto,0,IS_GENUINE,OEM:NONSLP,0,Retail,142,9414,0,0,0,0,0,1,6,2018-09-20 23:20:00,2018-09-11,1
3,38711eae85eb77a72ec5dfdf27eb2a76,win8defender,1.1.15200.1,4.12.17007.18011,1.275.1141.0,0,7,0,46413,2,1,1,68,19507,276,74,windows10,x64,10.0.0.0,15063,768,rs2,15063.0.amd64fre.rs2_release.170317-1834,Home,1,0,0,108,1,1,Notebook,Windows.Desktop,2102,242491,4,5,3410,1907729,HDD,1890776,0,8192,Notebook,15,1366,768,Mobile,0.000000e+00,10.0.15063.850,amd64,rs2_release,15063,850,Core,CORE,Upgrade,7,30,UNKNOWN,0,IS_GENUINE,OEM:DM,0,Retail,554,33060,1,0,0,0,0,0,12,2018-09-14 00:32:00,2018-01-03,1
4,32607c9a543a9214e2c7e45800ed4849,win8defender,1.1.15200.1,4.13.17134.228,1.275.1283.0,0,7,0,40466,2,1,1,43,117801,53,42,windows10,x86,10.0.0.0,17134,256,rs4,17134.1.x86fre.rs4_release.180410-1804,Pro,1,0,0,137,1,1,Desktop,Windows.Desktop,2668,257309,2,5,4322,305245,HDD,52804,0,2048,Desktop,20,1600,900,Desktop,4.294967e+09,10.0.17134.285,x86,rs4_release,17134,285,Professional,PROFESSIONAL,UUPUpgrade,37,158,FullAuto,0,IS_GENUINE,Retail,0,Retail,628,13224,0,0,0,0,0,1,7,2018-09-15 19:34:00,2018-09-11,0


In [127]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
IsBetaUser,100000.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00
RealTimeProtectionState,100000.0,6.843250e+00,1.034655e+00,-1.0,7.0,7.0,7.000000e+00,8.000000e+00
IsPassiveModeEnabled,100000.0,1.762000e-02,1.315664e-01,0.0,0.0,0.0,0.000000e+00,1.000000e+00
AntivirusConfigID,100000.0,4.793925e+04,1.386127e+04,-1.0,49480.0,53447.0,5.344700e+04,7.049000e+04
NumAntivirusProductsInstalled,100000.0,1.324760e+00,5.244175e-01,-1.0,1.0,1.0,2.000000e+00,5.000000e+00
NumAntivirusProductsEnabled,100000.0,1.016730e+00,1.648951e-01,-1.0,1.0,1.0,1.000000e+00,4.000000e+00
HasTpm,100000.0,9.967800e-01,5.665389e-02,0.0,1.0,1.0,1.000000e+00,1.000000e+00
CountryID,100000.0,1.080788e+02,6.306215e+01,1.0,51.0,97.0,1.620000e+02,2.220000e+02
CityID,100000.0,8.052512e+04,4.920615e+04,-1.0,35625.0,82373.0,1.225470e+05,1.679570e+05
GeoRegionID,100000.0,1.697416e+02,8.918893e+01,1.0,89.0,181.0,2.670000e+02,2.960000e+02


In [139]:
df_train.isnull().sum().sum()

np.int64(0)

In [7]:
df_train.shape

(100000, 76)

In [10]:
df_train.columns

Index(['MachineID', 'ProductName', 'EngineVersion', 'AppVersion',
       'SignatureVersion', 'IsBetaUser', 'RealTimeProtectionState',
       'IsPassiveModeEnabled', 'AntivirusConfigID',
       'NumAntivirusProductsInstalled', 'NumAntivirusProductsEnabled',
       'HasTpm', 'CountryID', 'CityID', 'GeoRegionID', 'LocaleEnglishNameID',
       'PlatformType', 'Processor', 'OSVersion', 'OSBuildNumber',
       'OSProductSuite', 'OsPlatformSubRelease', 'OSBuildLab',
       'SKUEditionName', 'IsSystemProtected', 'AutoSampleSubmissionEnabled',
       'SMode', 'IEVersionID', 'FirewallEnabled', 'EnableLUA',
       'MDC2FormFactor', 'DeviceFamily', 'OEMNameID', 'OEMModelID',
       'ProcessorCoreCount', 'ProcessorManufacturerID', 'ProcessorModelID',
       'PrimaryDiskCapacityMB', 'PrimaryDiskType', 'SystemVolumeCapacityMB',
       'HasOpticalDiskDrive', 'TotalPhysicalRAMMB', 'ChassisType',
       'PrimaryDisplayDiagonalInches', 'PrimaryDisplayResolutionHorizontal',
       'PrimaryDisplayResolutio

In [138]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 76 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   MachineID                           100000 non-null  object 
 1   ProductName                         100000 non-null  object 
 2   EngineVersion                       100000 non-null  object 
 3   AppVersion                          100000 non-null  object 
 4   SignatureVersion                    100000 non-null  object 
 5   IsBetaUser                          100000 non-null  int64  
 6   RealTimeProtectionState             100000 non-null  int64  
 7   IsPassiveModeEnabled                100000 non-null  int64  
 8   AntivirusConfigID                   100000 non-null  int64  
 9   NumAntivirusProductsInstalled       100000 non-null  int64  
 10  NumAntivirusProductsEnabled         100000 non-null  int64  
 11  HasTpm                     

In [45]:

object_col=df_train.select_dtypes(include=['object'])
intcol=df_train.select_dtypes(include=['int64','float64'])


In [130]:
df_train[object_col.columns] = df_train[object_col.columns].fillna(df_train[object_col.columns].mode().iloc[0])

In [131]:
df_train[intcol.columns] = df_train[intcol.columns].fillna(df_train[intcol.columns].mean()) 

In [49]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Scale numerical features
numerical_columns = df_train.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
df_train[numerical_columns] = scaler.fit_transform(df_train[numerical_columns])

# Handle missing values (already filled as per your query)
# Example: Fill missing values with mean for numerical columns
# dataset.fillna(dataset.mean(), inplace=True)




In [51]:
categorical_columns = df_train.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    df_train[col] = label_encoders[col].fit_transform(df_train[col])

In [132]:
# Step 1: Identify Binary Ordinal Features
binary_features = [col for col in df_train.columns if df[col].nunique() == 2]

# Step 2: Identify Ordinal Features (discrete numeric features with small integer values)
ordinal_features = []
for col in df_train.columns:
    if df[col].dtype in ['int64']: # Check for numeric columns
        unique_values = df[col].unique()
        if len(unique_values) > 2 and all(value == int(value) for value in unique_values): # Check if all values are integers
            ordinal_features.append(col)


In [125]:
# List of float columns to convert (replace with your actual column names)
float_columns = [
    'RealTimeProtectionState', 'AntivirusConfigID', 'NumAntivirusProductsInstalled', 
    'CityID', 'GeoRegionID', 'IsSystemProtected', 'SMode', 'IEVersionID', 
    'FirewallEnabled', 'EnableLUA', 'OEMNameID', 'OEMModelID',
    'ProcessorCoreCount', 'ProcessorManufacturerID', 'ProcessorModelID',
    'PrimaryDiskCapacityMB', 'SystemVolumeCapacityMB', 'TotalPhysicalRAMMB',
    'PrimaryDisplayDiagonalInches', 'PrimaryDisplayResolutionHorizontal',
    'PrimaryDisplayResolutionVertical', 'OSInstallLanguageID',
    'IsFlightsDisabled', 'FirmwareManufacturerID','NumAntivirusProductsEnabled','IsAlwaysOnAlwaysConnectedCapable', 
    'IsGamer','RegionIdentifier','IsVirtualDevice','FirmwareVersionID'

]

# Ensure all columns exist in the DataFrame
missing_columns = [col for col in float_columns if col not in df_train.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")
else:
    # Fill missing values and convert to int
    df_train[float_columns] = df_train[float_columns].fillna(-1).astype(int)
